In [3]:
import pandas as pd, numpy as np

In [4]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')

In [5]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [6]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]

In [7]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')

In [8]:
dmi=dm.loc[dc.index.dropna()]

In [9]:
df=dmi.merge(dc).drop_duplicates()

Working df ready. Now infer location.

In [10]:
locs=df[['category','icategory','tags','title','content']]

In [11]:
locs

,category,icategory,tags,title,content
0,aktualis,csik,['Csíkszentkirály'],Medvét láttak Csíkszentkirályon – ez volt az é...,Csíkszentkirály utcáin ólálkodó medvére hívta ...
1,aktualis,agerpres,"['Nézőpont', 'Székelyhon Tv']",Miért jár a medve iskolába? – Tánczos Barna a ...,"Az egyik legösszetettebb ágazat, a környezetvé..."
2,aktualis,agerpres,['Gyergyószék'],"Medvebocsokat szedtek ki egy barlangból, vide...","Eljárás indult négy Neamț megyei férfi ellen, ..."
3,aktualis,agerpres,"['Gyergyószék', 'medve']",Nem találták meg a medvebocsokat,Folytatódik a hatósági vizsgálat annak a négy ...
4,aktualis,csik,"['Háromszék', 'Környezetvédelem']",Mennyi medve van hat év semmittevés után?,Erre a kérdésre keresi első lépésben a választ...
5,aktualis,csik,"['Marosvásárhely', 'medve']",Elköltöztetik a Vásárhelyen sétáló medvét,Marosvásárhely két erdőszéli utcájában több al...
6,aktualis,udvarhely,"['Hargita megye', 'medve']","Újra aktívak a medvék, de még békén hagyják a...","Noha Hargita megyében már aktívak a medvék, a ..."
7,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Gyergyószentmiklósi bevásárlóközpontnál járt e...,Egy kis termetű medve felbukkanása miatt riasz...
8,aktualis,csik,['Gyergyószentmiklós'],Ismét medvét láttak Gyergyószentmiklóson,Medve jelenlétére hívták fel a hatóságok figye...
9,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Nem tágít Gyergyószentmiklósról a medve,Háromszor is riasztották a hatóságokat az elmú...


In [12]:
import json

In [13]:
lak=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [14]:
laks=[i for i in megyek.keys() if ((megyek[i]!='Más megye')and(i in lak.keys()))]

In [15]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [16]:
#look for perfect matches
decoded_locs={}
min_loc_len=5
for i in locs.index:
    if '[' in locs.loc[i]['tags']:
        k=locs.loc[i]['tags'].replace("'",'"')
        for j in json.loads(k):
            if j in laks:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(j)
                print('1',k,j)
    for j in locs.loc[i]['title'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('2',j)
    for j in locs.loc[i]['content'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('3',j)
    for j in locs.loc[i]['title'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('4',j,k)
    for j in locs.loc[i]['content'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('5',j,k)

1 ["Csíkszentkirály"] Csíkszentkirály
3 Csíkszentkirály
4 Csíkszentkirályon Csíkszentkirály
5 Csíkszentkirály Csíkszentkirály
5 Csíkszentkirály Csíkszentmihály
5 tavaly Haraly
5 nyomán Nyomát
5 keresi Keresd
1 ["Marosvásárhely", "medve"] Marosvásárhely
3 Marosvásárhely
5 Marosvásárhely Marosvásárhely
1 ["Gyergyószentmiklós", "medve"] Gyergyószentmiklós
4 Gyergyószentmiklósi Gyergyószentmiklós
5 Gyergyószentmiklóson Gyergyószentmiklós
5 kereső Kerelő
5 kereső Keresd
1 ["Gyergyószentmiklós"] Gyergyószentmiklós
4 Gyergyószentmiklóson Gyergyószentmiklós
5 Gyergyószentmiklóson Gyergyószentmiklós
1 ["Gyergyószentmiklós", "medve"] Gyergyószentmiklós
2 Ditró
5 Gyergyószentmiklósi Gyergyószentmiklós
5 Vadász Vadasd
5 magára Magura
5 szentegyházi Szentegyháza
4 Marosfőn Marosfő
5 Marosfőn, Marosfő
1 ["medve", "Sepsiszentgyörgy"] Sepsiszentgyörgy
3 Sepsiszentgyörgy
5 Sepsiszentgyörgy Sepsiszentgyörgy
3 Sepsiszentgyörgy
5 Sepsiszentgyörgy Sepsiszentgyörgy
1 ["medve", "Tusnádfürdő"] Tusnádfürdő
5 T

In [17]:
predicted_locs=[]
for i in range(len(locs.index)):
    if locs.index[i] in decoded_locs: 
        dummy=list(set(decoded_locs[locs.index[i]]))
        if len(dummy)==1:
            dummy=dummy[0]
        else:
            dummy=repr(list(set(decoded_locs[locs.index[i]])))
        predicted_locs.append(dummy)
    else: 
        predicted_locs.append('')

In [18]:
locs['predicted_locs']=predicted_locs
locs['actual_locs']=predicted_locs

<ipython-input-18-0909619aa9d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locs['predicted_locs']=predicted_locs
<ipython-input-18-0909619aa9d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locs['actual_locs']=predicted_locs


In [19]:
lm=locs[['title','content','predicted_locs','actual_locs']]

In [20]:
lm.to_excel('data/szekelyhon_locs.xlsx')
lm.to_excel('data/szekelyhon_locs_curated.xlsx')

Open `data/szekelyhon_locs_curated` and manually verify predicted locations. Amend `actual_locs` where necessary.